In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd


In [ ]:
import os
import gc
import sys
from PIL import Image
import cv2
import math, random
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

from torch.utils.data import DataLoader, Dataset

In [ ]:
train_df  = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train.csv')
train_label_cordi = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_label_coordinates.csv')
train_series_description = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv')

In [ ]:
#train_label_cordi.info()

In [ ]:
#train_series_description.info()

In [ ]:
train_df.info()

In [ ]:
#train_series_description

# Prepare data in dictionary format

In [ ]:
train_series_description

In [ ]:
train_label_cordi

In [ ]:
train_label_cordi['image_path'] = train_label_cordi.apply(lambda row: f"/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/{row['study_id']}/{row['series_id']}/{row['instance_number']}.dcm", axis=1)

In [ ]:
train_label_cordi = pd.merge(train_label_cordi, train_series_description, on=['study_id', 'series_id'], how='inner')

In [ ]:
train_label_cordi

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
train_label_cordi.iloc[0,:]

In [ ]:
result = {}
for study_id, group in train_label_cordi.groupby('study_id'):
    values = []
    for _, row in group.iterrows():
        values.append([
            row['series_description'],
            row['level'],
            row['x'],
            row['y'],
            row['image_path']
        ])
    result[f"{study_id}"] = values



In [ ]:
result.keys

### Label encoding

In [ ]:
train_df = train_df.fillna(-100)

In [ ]:
label2id = {'Normal/Mild': 0, 'Moderate':1, 'Severe':2}
train_df = train_df.replace(label2id)
train_df.head()

In [ ]:
#df = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train.csv')

In [ ]:
#df.head()

In [ ]:
class RSNA24Dataset(Dataset):
    def __init__(self, df,dc phase='train', transform=None):
        self.df = df
        self.transform = transform
        self.phase = phase
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        x = np.zeros((IN_CHANS,IMG_SIZE[0], IMG_SIZE[1], ), dtype=np.uint8)
        t = self.df.iloc[idx]
        st_id = int(t['study_id'])
        label = t[1:].values.astype(np.int64)
        
        # Sagittal T1
        for i in range(0, 10, 1):
            try:
                p = f'./cvt_png/{st_id}/Sagittal T1/{i:03d}.png'
                img = Image.open(p).convert('L')
                img = np.array(img)
                x[..., i] = img.astype(np.uint8)
            except:
                #print(f'failed to load on {st_id}, Sagittal T1')
                pass
            
        # Sagittal T2/STIR
        for i in range(0, 10, 1):
            try:
                p = f'./cvt_png/{st_id}/Sagittal T2_STIR/{i:03d}.png'
                img = Image.open(p).convert('L')
                img = np.array(img)
                x[..., i+10] = img.astype(np.uint8)
            except:
                #print(f'failed to load on {st_id}, Sagittal T2/STIR')
                pass
            
        # Axial T2
        axt2 = glob(f'./cvt_png/{st_id}/Axial T2/*.png')
        axt2 = sorted(axt2)
    
        step = len(axt2) / 10.0
        st = len(axt2)/2.0 - 4.0*step
        end = len(axt2)+0.0001
                
        for i, j in enumerate(np.arange(st, end, step)):
            try:
                p = axt2[max(0, int((j-0.5001).round()))]
                img = Image.open(p).convert('L')
                img = np.array(img)
                x[..., i+20] = img.astype(np.uint8)
            except:
                #print(f'failed to load on {st_id}, Sagittal T2/STIR')
                pass  
            
        assert np.sum(x)>0
            
        if self.transform is not None:
            x = self.transform(image=x)['image']

        x = x.transpose(2, 0, 1)
                
        return x, label

In [ ]:
grouped_counts = train_label_cordi.groupby('study_id').size().reset_index(name='Counts')

In [ ]:
missingCords = grouped_counts[grouped_counts['Counts'] < 25]

In [ ]:
pd.set_option('display.max_rows', 30)

In [ ]:
missingCords

In [ ]:
missing_cord_cordinates = train_label_cordi[train_label_cordi['study_id'].isin(missingCords['study_id'].values)]

In [ ]:
series_grouped_counts = missing_cord_cordinates.groupby('series_id').agg(
    study_id=('study_id', 'first'),  # example: take the first value of 'column1' in each group
    series_Counts=('series_id', 'size'),
    series_id=('series_id', 'first')
).reset_index(drop=True)

In [ ]:
series_grouped_counts = missing_cord_cordinates.groupby('series_id').size().reset_index(name='series_Counts')

In [ ]:
sorted_series_grouped_counts = series_grouped_counts.sort_values(by='study_id')

In [ ]:
sorted_series_grouped_counts

In [ ]:
missing_cord_des = train_series_description[train_series_description['series_id'].isin(sorted_series_grouped_counts['series_id'].values)]

In [ ]:
missing_cord_des

In [ ]:
merged_desc_df = pd.merge(sorted_series_grouped_counts, missing_cord_des, on=['study_id', 'series_id'], how='inner')

In [ ]:
merged_desc_df

In [ ]:
merged_desc_df[(merged_desc_df['series_description'] == 'Sagittal T2/STIR') & (merged_desc_df['series_Counts'] < 5) ]

In [ ]:
conditions = {
    'Sagittal T1': 10,
    'Axial T2': 10,
    'Sagittal T2/STIR': 5
}

# Initialize lists to store results
categories = []
meets_condition_counts = []
does_not_meet_condition_counts = []

# Iterate over the conditions and count the rows meeting and not meeting each condition
for category, threshold in conditions.items():
    meets_condition = merged_desc_df[(merged_desc_df['series_description'] == category) & (merged_desc_df['series_Counts'] < threshold)].shape[0]
    does_not_meet_condition = merged_desc_df[(merged_desc_df['series_description'] == category) & (merged_desc_df['series_Counts'] >= threshold)].shape[0]
    
    categories.append(category)
    meets_condition_counts.append(meets_condition)
    does_not_meet_condition_counts.append(does_not_meet_condition)

# Create a result DataFrame
result_df = pd.DataFrame({
    'Category': categories,
    'Meets Condition': meets_condition_counts,
    'Does Not Meet Condition': does_not_meet_condition_counts
})

In [ ]:
result_df

In [ ]:
train_df[train_df['study_id']== 4232806580 ]

In [ ]:
train_label_cordi[(train_label_cordi['instance_number'] != 8)  & (train_label_cordi['condition'] == 'Spinal Canal Stenosis') ]

In [ ]:
train_label_cordi[(train_label_cordi['study_id'] == 4232806580) ]

In [ ]:
import os

In [ ]:
files = os.listdir('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/46494080/1543341132')

In [ ]:
files

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import pydicom as dicom
import pydicom

In [ ]:
import glob
from matplotlib import animation, rc

rc('animation', html='jshtml')

def load_dicom(filename):
    ds = pydicom.dcmread(filename)
    return ds.pixel_array

def load_dicom_line(path):
    t_paths = sorted(
        glob.glob(os.path.join(path, "*")), 
        key=lambda x: int(os.path.splitext(os.path.basename(x))[0].split("-")[-1]),
    )
    images = []
    for filename in t_paths:
        data = load_dicom(filename)
        if data.max() == 0:
            continue
        images.append(data)
    return images

def create_animation(ims):
    fig = plt.figure(figsize=(6, 6))
    plt.axis('off')
    im = plt.imshow(ims[0], cmap="gray")

    def animate_func(i):
        im.set_array(ims[i])
        return [im]

    return animation.FuncAnimation(fig, animate_func, frames=len(ims), interval=1000//24)

path_to_folder = "/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/4232806580/3008065164"
images = load_dicom_line(path_to_folder)
anim = create_animation(images)
anim